In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option("display.precision", 4)

In [37]:
purchases = pd.read_csv('./purchases', sep='\t', decimal=',', parse_dates=['buy_ts'])
visits = pd.read_csv('./visits', sep='\t', decimal=',', parse_dates=['end_ts', 'start_ts'])

In [38]:
visits.drop(['start_dt'], axis=1 , inplace=True)

print(visits.info())
visits.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720679 entries, 0 to 720678
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   device     720679 non-null  object             
 1   end_ts     720679 non-null  datetime64[ns, UTC]
 2   source_id  720679 non-null  int64              
 3   start_ts   720679 non-null  datetime64[ns, UTC]
 4   uid        720679 non-null  int64              
dtypes: datetime64[ns, UTC](2), int64(2), object(1)
memory usage: 27.5+ MB
None


,device,end_ts,source_id,start_ts,uid
0,desktop,2016-11-30 11:04:17+00:00,2,2016-11-30 10:53:49+00:00,3758486201
1,desktop,2016-11-10 23:16:57+00:00,4,2016-11-10 23:08:04+00:00,2933941628
2,desktop,2016-10-19 18:51:56+00:00,1,2016-10-19 18:48:16+00:00,1465872791
3,desktop,2017-01-12 20:51:55+00:00,9,2017-01-12 20:50:10+00:00,2751875787
4,touch,2016-07-23 19:17:19+00:00,4,2016-07-23 19:10:57+00:00,3492372810


In [39]:
print(purchases.info())
purchases.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75716 entries, 0 to 75715
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   75716 non-null  datetime64[ns]
 1   uid      75716 non-null  int64         
 2   revenue  75716 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.7 MB
None


,buy_ts,uid,revenue
0,2016-07-23 19:13:54,3492372810,1.47
1,2016-06-11 02:48:27,1564753091,1.11
2,2017-01-31 20:32:42,1385616391,16.80
3,2017-11-01 10:52:11,2636761148,11.00
4,2016-11-25 18:57:09,2945629974,4.89


In [40]:
# Определяем, за какой временной промежуток были собраны данные
v_obs_ts = visits['end_ts'].max() - visits['start_ts'].min()
p_obs_ts = purchases['buy_ts'].max() - purchases['buy_ts'].min()
v_obs_ts, p_obs_ts
# => Visits за год, а purchases за два года

(Timedelta('365 days 01:26:41'), Timedelta('699 days 23:12:07'))

In [41]:
# Найдём самые базовые значения
total_revenue = purchases['revenue'].sum()
total_trans = len(purchases.index)
aov = total_revenue / total_trans

# Так как данных по gross margin нет, то предположим, что он равен 0.4
gm = 0.4

In [42]:
# Используем gross margin per customer для сверки с ltv, так как эти значения должны быть похожи
num_of_customers = purchases['uid'].nunique()
gm_per_customer = total_revenue / num_of_customers * gm
gm_per_customer

2.6585229953041547

In [43]:
# Отсортируем датафреймы по дате
purchases.sort_values(by='buy_ts', inplace=True)
visits.sort_values(by='start_ts', inplace=True)
purchases.reset_index(inplace=True)
visits.reset_index(inplace=True)

# В purchases добавим колонку new_uid, показывающую, является ли покупатель,
# совершивший покупку новым (до этого не совершал покупок)

customers_uid_set = set()
purchases['new_uid'] = purchases['uid']
for i in tqdm(range(len(purchases['uid']))):
    uid = purchases['new_uid'].iloc[i]
    purchases['new_uid'].iloc[i] = 0 if uid in customers_uid_set else 1
    customers_uid_set.add(uid)

visits_uid_set = set()
visits['new_uid'] = visits['uid']
for i in tqdm(range(len(visits['uid']))):
    uid = visits['new_uid'].iloc[i]
    visits['new_uid'].iloc[i] = 0 if uid in visits_uid_set else 1
    visits_uid_set.add(uid)

  0%|          | 0/75716 [00:00<?, ?it/s]d:\projects\olimps\python\venv\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 720679/720679 [07:17<00:00, 1648.47it/s]


In [44]:
# Произведём групировку покупок по месяцам и вынесем некоторые данные в отдельный датафрейм m_df

m_gdf = purchases.groupby(pd.Grouper(freq="M", key='buy_ts'))
m_df = pd.DataFrame()

# Колонка purchases показывает общее число покупок, совершённых в этом месяце
m_df['purchases'] = m_gdf['uid'].count().values

# Колонка first_purchases/unique_customers показывает одновременно и количество уникальных покупателей в этот месяц,
# и количество покупок, которые являются первыми у покупателей в этом месяце
m_df['first_purchases/uc'] = m_gdf['uid'].nunique().values

# Колонка repeating_purchases показывает количество повторных покупок в этом месяце
m_df['repeating_purchases'] = m_df['purchases'] - m_df['first_purchases/uc']

# Колонка new_customers показывает количество покупателей, которые впервые совершили покупку
# (не только в этом месяце, а за весь наблюдаемый период)
m_df['new_customers'] = m_gdf['new_uid'].sum().values

# Колонки показывает соответствующие показатели (retention rate и churn rate)
m_df['retention'] = (m_df['first_purchases/uc'] - m_df['new_customers']) / m_df['first_purchases/uc'].shift(1)
m_df['churn'] = (m_df['first_purchases/uc'].shift(1) - m_df['first_purchases/uc']) / m_df['first_purchases/uc'].shift(1)

# Не работаем с первыми тремя месяцами наблюдения для чистоты данных
m_df = m_df[3:]
m_df

,purchases,first_purchases/uc,repeating_purchases,new_customers,retention,churn
3,1323,1198,125,1094,0.0852,0.0188
4,1582,1417,165,1278,0.1160,-0.1828
5,3501,3118,383,2924,0.1369,-1.2004
6,3984,3486,498,3214,0.0872,-0.1180
7,3388,2954,434,2624,0.0947,0.1526
8,4947,4140,807,3701,0.1486,-0.4015
9,5873,4952,921,4336,0.1488,-0.1961
10,7138,5768,1370,4969,0.1613,-0.1648
11,6673,5548,1125,4845,0.1219,0.0381
12,5617,4671,946,4118,0.0997,0.1581


In [45]:
# Проверяем возможность использования churn rate для подсчёта ltv
churn = m_df['churn'].mean()
churn
# Отрицательный churn rate => Его нельзя использовать из-за того, что для его формулы подсчёта нет разницы
# между только что превлечёнными покупателями и уже сущестующими

# Исходя из визуального анализа данных, происходит постоянное превличение новых покупателей,
# однако старые в большенстве своём уходят сразу после первой покупки =>
# Для корректного подсчёта ltv в данной сетуации должен подойти retention rate

-0.06096150289553554

In [46]:
# Посчитаем purchases per month per customer (среднее количество покупок одного покупателя в месяц)
pmc = (((m_df['first_purchases/uc'] - m_df['repeating_purchases']) + m_df['repeating_purchases'] * 2) / (m_df['first_purchases/uc'])).mean()
retention = m_df['retention'].mean()
discount = 0.1 # Возьмём стандартное значение discount rate

# Average Revenue Per Paying User
arppu = pmc * aov * gm
clv = arppu * (1 + retention / (1 + discount - retention))
clv

2.6540437293800903

In [47]:
print('CLV: ', clv)
print('GMC: ', gm_per_customer)

# Полученое значение clv приближено к почитаному ранее gross margin per customer
# Это подтверждает верность полученного clv

CLV:  2.6540437293800903
GMC:  2.6585229953041547


In [48]:
# Далее анализ показателей для улучшения ltv

# Считаем среднее количество визитов и покупок для всех покупателей
# При этом фиксируем источник трафика, из которого пришёл конкретный покупатель

c_df = pd.DataFrame(index=customers_uid_set)
c_df['visits'] = np.zeros(len(customers_uid_set))
c_df['purchases'] = np.zeros(len(customers_uid_set))
c_df['first_source'] = np.zeros(len(customers_uid_set))

for p_uid in tqdm(purchases['uid'].values):
    u_visits = visits[visits['uid'] == p_uid]
    new_p = c_df.loc[p_uid, 'purchases'] + 1
    c_df.loc[p_uid, 'purchases'] = new_p

    if new_p != 1:
        continue

    c_df.loc[p_uid, 'visits'] = len(u_visits.index)
    c_df.loc[p_uid, 'first_source'] = u_visits['source_id'].values[0]

100%|██████████| 75716/75716 [02:48<00:00, 450.42it/s]


In [49]:
c_visits_mean = c_df['visits'].mean()
u_visits_mean = len(visits.index) / visits['uid'].nunique()

# Сравниваем visits покупателей и всех пользователей
print('Avr visits user: ', u_visits_mean)
print('Avr visits cust: ', c_visits_mean)

# Теперь смотрим на источники трафика по покупателям
c_df['first_source'].value_counts()

Avr visits user:  1.8959199833737153
Avr visits cust:  3.916370935942933


3.0     17847
4.0     16670
5.0     10369
2.0      4543
1.0      2561
10.0     2203
9.0      1598
7.0         3
Name: first_source, dtype: int64

In [50]:
# Источники трафика по всем пользователям
visits[visits['new_uid'] == 1]['source_id'].value_counts()

4     123898
3     116721
5      84386
2      24801
10     11941
9       9239
1       9062
7         70
6          3
Name: source_id, dtype: int64

In [51]:
# Посчитаем конверсию в покупателя для каждого источника трафика

def print_source_conv(n):
    a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
    print(n, ':', len(c_df[c_df['first_source'] == n].index)
          / (a if a != 0 else 1))

for i in range(11):
    print_source_conv(i)

# Видим, что конверсия во втором, первом и десятом источниках трафика наибольшая,
# Стоит вложить маркетинговые ресурсы именно туда

<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  a = len(visits[visits['new_uid'] == 1][visits['source_id'] == n].index)
<ipython-input-51-2c3c90ed9043>:4: UserWarning: Boolean Series key will be 

0 : 0.0
1 : 0.2826086956521739
2 : 0.18317809765735252
3 : 0.1529030765671987
4 : 0.13454615893719027
5 : 0.12287583248406134
6 : 0.0
7 : 0.04285714285714286
8 : 0.0
9 : 0.1729624418227081
10 : 0.1844904111883427


In [52]:
# Те же параметры с группировкой по источникам трафика

c_df.groupby(by='first_source').mean()

# Наиболее лояльные покупатели из второго источника =>
# Развивать второй источник

,visits,purchases
first_source,,
1.0,5.9613,1.6435
2.0,7.5294,2.0352
3.0,3.5908,1.2753
4.0,3.3512,1.2451
5.0,3.2192,1.3344
7.0,1.6667,1.0000
9.0,5.8761,1.4581
10.0,2.8652,1.1693
